In [14]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [16]:
class gineConv(torch.nn.Module):
    def __init__(self, in_channels, num_classes, edge_dim):
        super().__init__()

        self.nn_1 = torch.nn.Sequential(
            torch.nn.Linear(in_features=in_channels, out_features=20, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20)
        )

        self.nn_2 = torch.nn.Sequential(
            torch.nn.Linear(in_features=20, out_features=30, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, num_classes)
        )

        self.gine_conv1 = torch_geometric.nn.GINEConv(nn=self.nn_1, train_eps=True, edge_dim = edge_dim)
        self.gine_conv2 = torch_geometric.nn.GINEConv(nn=self.nn_2, train_eps=True, edge_dim = edge_dim)

    
    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.gine_conv1(x, edge_index, edge_attr)
        x = torch.nn.functional.relu(x)

        x = self.gine_conv2(x, edge_index, edge_attr)

        return torch.nn.functional.log_softmax(x, dim=1) #to use with loss function nll_loss

In [17]:
def train(loader, model, loss_fn, optimizer):
    total_num_dataset = len(loader.dataset)
    model.train()
    for batch_nr, batch_dat in loader:
        batch_dat = batch_dat.to(device)

        pred = model(batch_dat)
        loss = loss_fn(pred, batch_dat.y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_nr % 100 == 0:
            loss, current = loss.item(), (batch_nr + 1)*len(batch_dat)
            print(f"loss: {loss:>7f} [{current:>5d}/{total_num_dataset:>5d}]")

In [18]:
model = gineConv(10, 5, 10)
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

#dataset = [Data(), Data(), Data(), ...]
#train_dataloader = DataLoader(dataset, batch_size=, shuffle=)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------")
    #train(train_dataloader, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)
print("Done")


Epoch 1
----------------------------------------
Epoch 2
----------------------------------------
Epoch 3
----------------------------------------
Epoch 4
----------------------------------------
Epoch 5
----------------------------------------
Done
